## Raster Data

In scikit-map, the class RasterData is responsible for implement reading, processing and writing operations. The library provides a NDVI toy dataset that you can access by:

In [123]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
sys.path.insert(0, os.path.abspath('../../'))
from skmap.data import toy
from skmap.io import process
import importlib
importlib.reload(process)
data = np.random.rand(1,2,15)
sircleTransformer = process.SircleTransformer(w_0=1, w_f=np.array([0.5, 0.5]), w_p=np.array([2, 1, 1]), backend="dense")
out_dense = sircleTransformer._run(data)
sircleTransformer = process.SircleTransformer(w_0=1, w_f=np.array([0.5, 0.5]), w_p=np.array([2, 1, 1]), backend="sparse")
out_sparse = sircleTransformer._run(data)
sircleTransformer = process.SircleTransformer(w_0=1, w_f=np.array([0.5, 0.5]), w_p=np.array([2, 1, 1]), backend="FFT")
out_fft = sircleTransformer._run(data)
sircleTransformer = process.SircleTransformer(w_0=1, w_f=np.array([0.5, 0.5]), w_p=np.array([2, 1, 1]), backend="iterative")
out_iter = sircleTransformer._run(data)




Same but with no future images

In [124]:
V_e = np.zeros((10, 3), dtype=np.float64, order='F')

In [125]:
out_dense[0,1,:]

array([0.75769417, 1.42400666, 1.74734931, 2.78023693, 2.98670158,
       3.23069915, 3.31752645, 2.33775447, 3.61574512, 2.55791506,
       3.56400742, 3.11846102, 2.91667801, 3.49702068, 2.19795098])

In [126]:
out_fft[0,1,:]

array([0.75769417, 1.42400666, 1.74734931, 2.78023693, 2.98670158,
       3.23069915, 3.31752645, 2.33775447, 3.61574512, 2.55791506,
       3.56400742, 3.11846102, 2.91667801, 3.49702068, 2.19795098])

In [127]:
out_sparse[0,:,:]

array([[0.85622671, 1.18990307, 1.61796426, 2.86632611, 2.79279626,
        2.73910748, 3.70618478, 4.21706942, 4.52980163, 4.00102499,
        3.22771855, 4.42825835, 3.03251076, 3.6295353 , 2.96302395],
       [0.75769417, 1.42400666, 1.74734931, 2.78023693, 2.98670158,
        3.23069915, 3.31752645, 2.33775447, 3.61574512, 2.55791506,
        3.56400742, 3.11846102, 2.91667801, 3.49702068, 2.19795098]])

In [128]:
out_iter[0,:,:]

array([[0.85622671, 1.18990307, 1.61796426, 2.86632611, 2.79279626,
        2.73910748, 3.70618478, 4.21706942, 4.52980163, 4.00102499,
        3.22771855, 4.42825835, 3.03251076, 3.6295353 , 2.96302395],
       [0.75769417, 1.42400666, 1.74734931, 2.78023693, 2.98670158,
        3.23069915, 3.31752645, 2.33775447, 3.61574512, 2.55791506,
        3.56400742, 3.11846102, 2.91667801, 3.49702068, 2.19795098]])

In [110]:
import numpy as np 
from scipy.linalg import circulant
from numpy.fft import fft, ifft

def circulant_matrix_vector_product(v, w):
    # Compute FFT of v and padded_w
    W = circulant(w)
    
    fft_v = fft(v)
    fft_w = fft(W[0,:])
    
    # Element-wise multiplication in the frequency domain
    fft_result = fft_v * fft_w
    
    # Inverse FFT to get the result in the time domain
    result = ifft(fft_result)
    
    return result.real  # Take the real part to handle numerical precision issues

def circulant_matrix_vector_product_normal(v, w):
#     n = len(v)
#     m = len(w)
#     if n != m:
#         padded_w = np.concatenate((w, np.zeros(n - m)))
#     else:
#         padded_w = w
    
    # Create circulant matrix W
    W = circulant(w)
    
    # Matrix-vector multiplication
    result_normal = np.dot(v,W)
    
    return result_normal

# Example usage:
n_s = data.shape[2]
n_e = n_s + max(3, 2)
V_e = np.zeros((1,n_e), dtype=np.float64, order='F')
V_e[:, 0:n_s] = data
v = V_e[0,:]
w = np.zeros((n_e,))
w[0] = 1
w[1:2+1] = np.array([0.5, 0.5])
w[-3:] = np.array([2, 1, 1])
result_fft = circulant_matrix_vector_product(v, w)
result_normal = circulant_matrix_vector_product_normal(v, w)

# Compare the results
print("FFT Result:", result_fft)
print("Normal Result:", result_normal)
print("Results are equal:", np.allclose(result_fft, result_normal))

FFT Result: [1.64445915 2.38651523 3.26200896 4.98402487 4.2551079  4.3747972
 3.76027619 3.32954527 2.9531219  1.95083089 2.7680308  3.37976723
 4.01446581 3.71763899 2.86477876 3.18611245 2.85165612 2.58671261]
Normal Result: [1.64445915 2.38651523 3.26200896 4.98402487 4.2551079  4.3747972
 3.76027619 3.32954527 2.9531219  1.95083089 2.7680308  3.37976723
 4.01446581 3.71763899 2.86477876 3.18611245 2.85165612 2.58671261]
Results are equal: True
